# Problema 1: Distribución Óptima de Recursos en una Misión Humanitaria

## 1. Preprocesamiento de datos
Los datos se van a ajustar a escala para así poder modelar que los recursos son divisibles. Entonces, vamos a convertir los valores de los pesos a kg, para así homogenizar los datos. Entonces, en el modelo va a ser facil manejar la variable $X_{ij}$, que puede representar cuántos kg del recurso r se va a transportar en el avion j. Adicionalmente, es de relevancia saber el _valor_ y el _volumen_ por kg. Es decir, un kilo de cada recurso ¿cuánto vale?, ¿cuánto es su volumen? Para poder calcular esto, se va a dividir el valor y volumen, cada uno entre el peso en kg.
##### Valor en KG
$Va_{kg} = \frac{ValorRecurso}{Peso (KG)} $

##### Volumen en KG
$Vo_{kg}= \frac{VolumenRecurso}{Peso (KG)} $ 

### Valores originales:
#### Recursos 
| Recurso           | Valor | Peso (TON) | Volumen (m³) |
|-------------------|:-----:|:----------:|:------------:|
| Alimentos Básicos |   50  |     15     |      8       |
| Medicinas         |  100  |      5     |      2       |
| Equipos Médicos   |  120  |     20     |     10       |
| Agua Potable      |   60  |     18     |     12       |
| Mantas            |   40  |     10     |      6       |

#### Aviones
| Avión | Capacidad (TON) | Capacidad (m³) |
|-------|:---------------:|:--------------:|
|   1   |       30        |       25       |
|   2   |       40        |       30       |
|   3   |       50        |       35       |


### Valores nuevos en kg:  
#### Recursos 
| Recurso           | Valor (por KG) | Peso (KG) | Volumen (m³ por KG) |
|-------------------|:-------------:|:---------:|:-------------------:|
| Alimentos Básicos |   $\frac{50}{15000}$  |   15000   |   $\frac{8}{15000}$   |
| Medicinas         | $\frac{100}{5000}$  |    5000   |   $\frac{2}{5000}$    |
| Equipos Médicos   |  $\frac{120}{20000}$ |   20000   |   $\frac{10}{20000}$  |
| Agua Potable      |  $\frac{60}{18000}$  |   18000   |   $\frac{12}{18000}$  |
| Mantas            |  $\frac{40}{10000}$  |   10000   |   $\frac{6}{10000}$   |

#### Aviones
| Avión | Capacidad (KG) | Capacidad (m³ por KG) |
|-------|:--------------:|:---------------------:|
|   1   |     30000      | 25  |
|   2   |     40000      | 30  |
|   3   |     50000      | 35  |





## 2. Formulación del Modelo Matemático
### 1. Conjuntos
$R$, para los recursos esenciales que se necesitan transportar, donde $R={1,2,3,4,5}$. Con $i$ se va a indexar este conjunto. Los valores de $R$ son referencias a los recursos, donde
| Referencia | Recurso | 
|-------|:---------------:|
|   1   |Alimentos Básicos|
|   2   |Medicinas|
|   3   |Equipos médicos|
|   4   |Agua potable|
|   5   |Mantas|

$A$, para la flota de aviones, donde $A={1,2,3}$. Con $j$ se va a indexar este conjunto.
### 2. Parámetros
|Nombre|Descripcion|
|------|-----------|
|$VA_i$|El valor por cada kg del recurso $i$.|
|$VO_i$|El volumen de cada kg (en $m^3$) del recurso i|
|$P_i$|El peso (en kg) del recurso i|
|$CP_j$|La capacidad de peso (en kg) de cada avion|
|$CV_j$|La capacidad de volumen de cada (en $m^3$) de cada avion|

En estos datos de entrada, ya se realizan los cambios de TON a KG, como se explicó en el preprocesamiento de datos.

### 3. Variables de decisión

- $X_{ij}$ real, en la cual se indica para el recurso i, en el avion j, cuantos kg va a cargar.
- $Z_j$ entero, para el recurso 3 (Equipos Médicos), que son dispositivos indivisibles (300 kg c/u). Indica cuántos dispositivos de equipos médicos se llevan en el avión 𝑗. Cada dispositivo pesa 300 kg y ocupa $300* VO_3$ en volumen.
- $Y_{ij}$ binario, el cual indica si el recurso i lo lleva el avion j
### 4. Función objetivo
- $max(\sum_{i\in R, i!=3}^{}\sum_{j\in A}^{} VA_{i}*Y_{ij}+\sum{j\in A}300*VA_3 * Z_j)$
- Es importante mencionar que para $i!= 3$ el valor maximizado es el valor por kg, por la cantidsd de kg llevado. Sin embargo, para los equipos medicos, $i=3$, cada dispositivo "aporta" 300 por la cantidad de valor de kg de este recurso, por cada vez que sea llevado en algun avion.
### 5. Restricciones
#### 5.1. Seguridad de medicamentos
- $X_{2,1}=0$
#### 5.2. Compatibilidad entre equipos medicos y agua
- $Y_{3,j}+Y_{4,j} <=1 \forall j\in V$
#### 5.3. Capacidad de peso en los aviones
- $\sum_{i\in R, i!=3}^{}X_{ij} + 300 * Z_j <= CP_j \forall j\in A$

#### 5.4. Capacidad de volumen en los aviones
- $\sum_{i\in R, i!=3}^{}(X_{ij} * VO_i) +(300 * C0_3)* Z_j <= CV_j \forall j\in A$

#### 5.5. Cantidad de recursos no exceda el limite
- $\sum_{j\in A}^{}X_{ij}<= P_i \forall i\in R$

#### 5.6. Consistencia en variable de decision con variables numericas
- $Z_j <= M * Y_{3,j}\forall j\in A$
- $X_{i,j} <= M * Y_{i,j}\forall i\in R|i!=3 \forall j\in A$
- Para activar compatibilidad para vincular la variable binaria $Y_{3,j}$ con la variable entera de recurso 3 y demas; $Z_j$ y $X_{i,j}, respectivamente$.
- Para esto, se utiliza una Big M
### 6. Tipo de problema
MIP, ya que tenemos variables binarias y reales.

## 3. Implementación de Pyomo

In [ ]:
from __future__ import division
from pyomo.environ import *
from pyomo.opt import SolverFactory

Model = ConcreteModel()

# Data de entrada
numRecursos = 5
numAviones = 3

# Conjuntos
R = RangeSet(1, numRecursos)   
A = RangeSet(1, numAviones)    

# Parámetros
# i : 1=Alimentos, 2=Medicinas, 3=Equipos Médicos, 4=Agua, 5=Mantas
recursos = {
    1: {"valor": 50/15000,  "peso": 15000, "volumen": 8/15000, "nombre":"Alimentos"      },    
    2: {"valor": 100/5000,  "peso": 5000,  "volumen": 2/5000,  "nombre":"Medicinas"      },     
    3: {"valor": 120/20000, "peso": 20000, "volumen": 10/20000,"nombre":"Equipos Medicos"},   
    4: {"valor": 60/18000,  "peso": 18000, "volumen": 12/18000,"nombre":"Agua"           },   
    5: {"valor": 40/10000,  "peso": 10000, "volumen": 6/10000, "nombre":"Mantas"         },    
}

aviones = {
    1: {"capPeso": 30000, "capVolumen": 25},
    2: {"capPeso": 40000, "capVolumen": 30},
    3: {"capPeso": 50000, "capVolumen": 35}
}

#Datos especiales para Equipos Médicos (recurso 3)
medPeso = 300                           
medVolumen = medPeso * recursos[3]["volumen"]
medValor = medPeso * recursos[3]["valor"]
maxEquipos = recursos[3]["peso"] / medPeso  # Máximo número de dispositivos disponibles
# (Por ej., 20000 kg / 300 kg = 66.66 → 66 dispositivos)

#  Variables de Decisión
Model.x = Var(R, A, domain=NonNegativeReals)

Model.y = Var(R, A, domain=Binary)

Model.z = Var(A, domain=NonNegativeIntegers)

#  Función Objetivo
Model.obj = Objective(
    expr = sum(Model.x[i, j]*recursos[i]["valor"] 
               for i in R if i != 3 
               for j in A)
         + sum(medValor * Model.z[j] for j in A),
    sense=maximize
)

#  Restricciones

# 1 Seguridad de medicamentos
Model.rest1 = Constraint(expr = Model.x[2,1] == 0)

# 2 compatibilidad entre equipos medicos (3) y agua (4) 
Model.rest2 = ConstraintList()
for j in A:
    Model.rest2.add(Model.y[3,j] + Model.y[4,j] <= 1)

# 3 Capacidad de peso en aviones
Model.rest3 = ConstraintList()
for j in A:
    Model.rest3.add(
        sum(Model.x[i,j] for i in R if i != 3)
        + medPeso*Model.z[j]
        <= aviones[j]["capPeso"]
    )

# 4 Capacidad de volumen en aviones
Model.rest4 = ConstraintList()
for j in A:
    Model.rest4.add(
        sum(Model.x[i,j]*recursos[i]["volumen"] for i in R if i != 3)
        + medVolumen*Model.z[j]
        <= aviones[j]["capVolumen"]
    )

# 5 Cantidad de recurso no exceda el limite
Model.rest5 = ConstraintList()
for i in R:
    if i != 3:
        Model.rest5.add(
            sum(Model.x[i,j] for j in A) <= recursos[i]["peso"]
        )
    else:
        Model.rest5.add(
            sum(Model.z[j] for j in A) <= int(maxEquipos)
        )

# 6 - Consistencia entre x[i,j] y y[i,j]
Model.rest6 = ConstraintList()
M = 100000000000000
for i in R:
    if i != 3:
        for j in A:
            
            Model.rest6.add(Model.x[i,j] <= M * Model.y[i,j])
    elif i == 3:
        
            Model.rest6.add(Model.z[i] <= M * Model.y[i,j])
       
# Especificación  solver
SolverFactory('glpk').solve(Model)
Model.display()


Model unknown

  Variables:
    x : Size=15, Index=x_index
        Key    : Lower : Value   : Upper : Fixed : Stale : Domain
        (1, 1) :     0 :     0.0 :  None : False : False : NonNegativeReals
        (1, 2) :     0 : 15000.0 :  None : False : False : NonNegativeReals
        (1, 3) :     0 :     0.0 :  None : False : False : NonNegativeReals
        (2, 1) :     0 :     0.0 :  None : False : False : NonNegativeReals
        (2, 2) :     0 :  5000.0 :  None : False : False : NonNegativeReals
        (2, 3) :     0 :     0.0 :  None : False : False : NonNegativeReals
        (3, 1) :     0 :    None :  None : False :  True : NonNegativeReals
        (3, 2) :     0 :    None :  None : False :  True : NonNegativeReals
        (3, 3) :     0 :    None :  None : False :  True : NonNegativeReals
        (4, 1) :     0 : 18000.0 :  None : False : False : NonNegativeReals
        (4, 2) :     0 :     0.0 :  None : False : False : NonNegativeReals
        (4, 3) :     0 :     0.0 :  Non

In [ ]:
def reporte_solucion(model, R, A):
    print("\n=== SOLUCIÓN ===")
    # valor objetivo
    print(f"Valor total transportado: {value(model.obj):.4f}\n")
    
    # Pa cada avión
    for j in A:
        print(f"Avión {j}:")
        
        # recursos fraccionables (i != 3)
        for i in R:
            if i != 3:  
                cantidad = model.x[i,j].value
                if cantidad > 0: 
                    print(f"  - {recursos[i]['nombre']}: {cantidad:.2f} kg")
        
        # equipos médicos (recurso 3) en unidades y pesos
        dispositivos = model.z[j].value
        if dispositivos > 0: 
            print(f"  - Equipos Médicos: {int(dispositivos)} unidad(es), {int(dispositivos)*medPeso} kg")
        
        print() 

reporte_solucion(Model, R, A)


=== REPORTE DE SOLUCIÓN ===
Valor total transportado: 368.8000

Avión 1:
  - Agua: 18000.00 kg
  - Equipos Médicos: 40 unidad(es), 12000 kg

Avión 2:
  - Alimentos: 15000.00 kg
  - Medicinas: 5000.00 kg
  - Mantas: 10000.00 kg

Avión 3:
  - Equipos Médicos: 26 unidad(es), 7800 kg



## Analisis de resultados
El valor total transportado dio la forma más óptima. Sin embargo, aunque el modelo permitia que los recursos se dividieran, se encontró que en la forma más óptima solo se dividen los equipos médicos (recurso 3), pero el resto se transportan juntos. Se puede concluir que se encontró la forma más óptima para maximizar el valor de los recursos transportados.